In [1]:
import os
import json
import requests
#__location__ = os.path.realpath(
#    os.path.join(os.getcwd(), os.path.dirname(__file__)))

# THIS HANDLES NO ERRORS AND IS JUST A RAW CONNECTION TO THE LOL API;
# USE THE API MODULE TO GET ERROR FALLBACK METHODS

with open(os.path.join(__location__, 'config'), 'r') as f:
    try:
        config = json.load(f)
    except ValueError:
        config = {'key': ""}

KEY = config["key"]
GLOBAL_ENDPOINT = "https://global.api.pvp.net/api/lol/static-data/{}/"
REGION_ENDPOINT = "https://{0}.api.pvp.net/api/lol/{0}/"


def set_api_key(key):
    global KEY
    with open('config', 'w') as f:
        json.dump({"key": key}, f)
    KEY = key


# CHAMPION-v1.2


def get_champions(region, freetoplay="false"):
    """
    Retrieve all champions.
    """
    return requests.get(
        (REGION_ENDPOINT + "v1.2/champion?freeToPlay={1}&api_key={2}").
        format(region, freetoplay, KEY))


def get_champion(region, championId):
    """
    Retrieve champion by ID.
    """
    return requests.get(
        (REGION_ENDPOINT + "v1.2/champion/{1}?api_key={2}").
        format(region, championId, KEY))

# GAME-v1.3


def get_recent_games(region, summonerId):
    """
    Get recent games by summoner ID.
    """
    return requests.get(
        (REGION_ENDPOINT + "v1.3/game/by-summoner/{1}/recent?api_key={2}").
        format(region, summonerId, KEY))

# LEAGUE-v2.5


def get_league(region, summonerIds):
    """
    Get leagues mapped by summoner ID for a given list of summoner IDs.
    """
    return requests.get(
        (REGION_ENDPOINT + "v2.5/league/by-summoner/{1}?api_key={2}").
        format(region, summonerIds, KEY))


def get_league_entry(region, summonerIds):
    """
    Get league entries mapped by summoner ID
    for a given list of summoner IDs.
    """
    return requests.get(
        (REGION_ENDPOINT + "v2.5/league/by-summoner/{1}/entry?api_key={2}").
        format(region, summonerIds, KEY))


def get_league_by_team(region, teamIds):
    """
    Get leagues mapped by team ID for a given list of team IDs.
    """
    return requests.get(
        (REGION_ENDPOINT + "v2.5/league/by-team/{1}?api_key={2}").
        format(region, teamIds, KEY))


def get_league_entry_by_team(region, teamIds):
    """
    Get league entries by team ID for a given list of team IDs.
    """
    return requests.get(
        (REGION_ENDPOINT + "v2.5/league/by-team/{1}?api_key={2}").
        format(region, teamIds, KEY))


def get_challenger_league_tiers(region, queuetype):
    """
    Get challenger tier leagues.
    """
    return requests.get(
        (REGION_ENDPOINT + "v2.5/league/challenger?type={1}&api_key={2}").
        format(region, queuetype, KEY))

# LOL-STATIC-DATA-v1.2


def get_champion_list(
        region, locale="", version="",
        dataById="", champData=""):
    """
    Retrieves champion list.
    """
    return requests.get(
        (GLOBAL_ENDPOINT + "v1.2/champion?locale={1}&version={2}"
         "&dataById={3}&champData={4}&api_key={5}").
        format(region, locale, version, dataById, champData, KEY))


def get_champion_list_by_id(
        region, championId, locale="", version="", champData=""):
    """
    Retrieves a champion by its id.
    """
    return requests.get(
        (GLOBAL_ENDPOINT + "v1.2/champion/{1}?locale={2}"
         "&version={3}&champData={4}&api_key={5}")
        .format(region, championId, locale, version, champData, KEY))


def get_item_list(
        region, locale="", version="", itemListData=""):
    """
    Retrieves item list.
    """
    return requests.get(
        (GLOBAL_ENDPOINT + "v1.2/item?locale={1}&version={2}"
         "&itemListData={3}&api_key={4}").
        format(region, locale, version, itemListData, KEY))


def get_item_list_by_id(
        region, itemId, locale="", version="", itemData=""):
    """
    Retrieves item by its unique id.
    """
    return requests.get(
        (GLOBAL_ENDPOINT + "v1.2/item/{1}?locale={2}"
         "&version={3}&itemData={4}&api_key={5}")
        .format(region, itemId, locale, version, itemData, KEY))


def get_mastery_list(
        region, locale="", version="", masteryListData=""):
    """
    Retrieves mastery list.
    """
    return requests.get(
        (GLOBAL_ENDPOINT + "v1.2/mastery?locale={1}&version={2}"
         "&masteryListData={3}&api_key={4}").
        format(region, locale, version, masteryListData, KEY))


def get_mastery_list_by_id(
        region, masteryId, locale="", version="", masteryData=""):
    """
    Retrieves mastery item by its unique id.
    """
    return requests.get(
        (GLOBAL_ENDPOINT + "v1.2/item/{1}?locale={2}"
         "&version={3}&masteryData={4}&api_key={5}")
        .format(region, masteryId, locale, version, masteryData, KEY))


def get_realm_data(region):
    """
    Retrieves realm data.
    """
    return requests.get(
        (GLOBAL_ENDPOINT + "v1.2/realm?api_key={1}").
        format(region, KEY))


def get_rune_list(region, locale="", version="", runeListData=""):
    """
    Retrieves rune list.
    """
    return requests.get(
        (GLOBAL_ENDPOINT + "v1.2/rune?locale={1}&version={2}"
         "&runeListData={3}&api_key={4}").
        format(region, locale, version, runeListData, KEY))


def get_rune_list_by_id(
        region, runeId, locale="", version="", runeData=""):
    """
    Retrieves rune by its unique id.
    """
    return requests.get(
        (GLOBAL_ENDPOINT + "v1.2/rune/{}?locale={1}"
         "&version={2}&runeData={3}&api_key={4}")
        .format(region, runeId, locale, version, runeData, KEY))


def get_spell_list(
        region, locale="", version="",
        dataById="", spellData=""):
    """
    Retrieves summoner spell list.
    """
    return requests.get(
        (GLOBAL_ENDPOINT + "v1.2/summoner-spell?locale={1}&version={2}"
         "&dataById={3}&spellData={4}&api_key={5}").
        format(region, locale, version, dataById, spellData, KEY))


def get_spell_list_by_id(
        region, spellId, locale="", version="", spellData=""):
    """
    Retrieves summoner spell by its unique id.
    """
    return requests.get(
        (GLOBAL_ENDPOINT + "v1.2/summoner-spell/{1}?locale={2}"
         "&version={3}&spellData={4}&api_key={5}")
        .format(region, spellId, locale, version, spellData, KEY))


def get_version_data(region):
    """
    Retrieves version data.
    """
    return requests.get(
        (GLOBAL_ENDPOINT + "v1.2/versions?api_key={1}").
        format(region, KEY))

# LOL-STATUS-v1.0


def get_shards():
    """
    Get shard list.
    """
    return requests.get("http://status.leagueoflegends.com/shards")


def get_shard_status(region):
    """
    Get shard status.
    Returns the data available on the status.leagueoflegends.com website
    for given region.
    """
    return requests.get("http://status.leagueoflegends.com/shards/{}"
                        .format(region))

# MATCH-v2.2


def get_match(region, matchId, includeTimeline):
    """
    Retrieve match by match ID.
    """
    return requests.get(
        (REGION_ENDPOINT + "v2.2/match/{1}?"
         "api_key={2}&includeTimeline={3}").
        format(region, matchId, KEY, includeTimeline))

# MATCHHISTORY-v2.2


def get_matchhistory(region, summonerId):
    """
    Retrieve match history by summoner ID.
    """
    return requests.get(
        (REGION_ENDPOINT + "v2.2/matchhistory/{1}?"
         "api_key={2}").
        format(region, summonerId, KEY))

# STATS-v1.3


def get_ranked_stats(region, summonerId, season=""):
    """
    Get ranked stats by summoner ID.
    """
    return requests.get(
        (REGION_ENDPOINT + "v1.3/stats/by-summoner/{1}/ranked?"
         "api_key={2}&season={3}").
        format(region, summonerId, KEY, season))


def get_stats(region, summonerId, season=""):
    """
    Get player stats summaries by summoner ID.
    """
    return requests.get(
        (REGION_ENDPOINT +
         "v1.3/stats/by-summoner/{1}/summary?api_key={2}&season={3}").
        format(region, summonerId, KEY, season))

# SUMMONER-v1.4


def get_summoner_by_name(region, summonerNames):
    """
    Get summoner objects mapped by standardized summoner name
    for a given list of summoner names.
    """
    return requests.get(
        (REGION_ENDPOINT + "v1.4/summoner/by-name/{1}?&api_key={2}").
        format(region, summonerNames, KEY))


def get_summoner_by_id(region, summonerIds):
    """
    Get summoner objects mapped by summoner ID
    for a given list of summoner IDs.
    """
    return requests.get(
        (REGION_ENDPOINT + "v1.4/summoner/{1}?api_key={2}").
        format(region, summonerIds, KEY))


def get_masteries(region, summonerIds):
    """
    Get mastery pages mapped by summoner ID
    for a given list of summoner IDs.
    """
    return requests.get(
        (REGION_ENDPOINT + "v1.4/summoner/{1}/masteries?api_key={2}").
        format(region, summonerIds, KEY))


def get_name(region, summonerIds):
    """
    Get summoner names mapped by summoner ID
    for a given list of summoner IDs.
    """
    return requests.get(
        (REGION_ENDPOINT + "v1.4/summoner/{1}/name?api_key={2}").
        format(region, summonerIds, KEY))


def get_runes(region, summonerIds):
    """
    Get rune pages mapped by summoner ID
    for a given list of summoner IDs.
    """
    return requests.get(
        (REGION_ENDPOINT + "v1.4/summoner/{1}/runes?api_key={2}").
        format(region, summonerIds, KEY))

# TEAM-v2.4


def get_teams_by_summonerid(region, summonerIds):
    """
    Get teams mapped by summoner ID
    for a given list of summoner IDs.
    """
    return requests.get(
        (REGION_ENDPOINT + "v2.4/team/by-summoner/{1}?api_key={2}").
        format(region, summonerIds, KEY))


def get_teams(region, teamIds):
    """
    Get teams mapped by team ID
    for a given list of team IDs.
    """
    return requests.get(
        (REGION_ENDPOINT + "v2.4/team/{1}?api_key={2}").
        format(region, teamIds, KEY))

ImportError: No module named rawpi

In [1]:
import os
import json
import requests
#__location__ = os.path.realpath(
#    os.path.join(os.getcwd(), os.path.dirname(__file__)))

# THIS HANDLES NO ERRORS AND IS JUST A RAW CONNECTION TO THE LOL API;
# USE THE API MODULE TO GET ERROR FALLBACK METHODS

#with open(os.path.join(__location__, 'config'), 'r') as f:
#    try:
#        config = json.load(f)
#    except ValueError:
#        config = {'key': ""}

KEY = 'b00c992b-6fc1-4795-abcc-7db85c1b72fa'

REGION_ENDPOINT = "https://{0}.api.pvp.net/api/lol/{0}/"

def get_recent_games(region, summonerId):
    """
    Get recent games by summoner ID.
    """
    return requests.get(
        (REGION_ENDPOINT + "v1.3/game/by-summoner/{1}/recent?api_key={2}").
        format(region, summonerId, KEY))

def get_teams_by_summonerid(region, summonerIds):
    """
    Get teams mapped by summoner ID
    for a given list of summoner IDs.
    """
    return requests.get(
        (REGION_ENDPOINT + "v2.4/team/by-summoner/{1}?api_key={2}").
        format(region, summonerIds, KEY))

In [113]:
jaemin_recent_games = get_recent_games('na', 61281860)
jaemin_teams = get_teams_by_summonerid('na', 61281860)
#print jaemin_recent_games.text
#print jaemin_teams.text

In [3]:
# Champion ID to Champion Name converter
KEY = 'b00c992b-6fc1-4795-abcc-7db85c1b72fa'

# GET the static data on champion keys and names
static_champ_data = requests.get("https://global.api.pvp.net/api/lol/static-data/na/v1.2/champion?locale=en_US&api_key=b00c992b-6fc1-4795-abcc-7db85c1b72fa").json()
champs = static_champ_data['data']
champ_names = static_champ_data['data'].keys()

# initializing index and empty dictionaries
i = 0
list_champs = []
list_champ_ids = []

# filling in the lists
for i in range(len(champ_names)):
    this_champ = champs[champ_names[i]]
    list_champs.append(this_champ['key'])
    list_champ_ids.append(this_champ['id'])
    
# champion ID and name dictionary    
champ_dict = dict(zip(list_champ_ids, list_champs))

# Wukong's name is not correct (MonkeyKing)
champ_dict[62] = 'Wukong'

# function that converts a champion ID to champion name
def champion_name(champion_id):
    if champion_id in champ_dict:
        return champ_dict[champion_id]
    else:
        return 'no_name'


In [4]:
champ_dict

{1: u'Annie',
 2: u'Olaf',
 3: u'Galio',
 4: u'TwistedFate',
 5: u'XinZhao',
 6: u'Urgot',
 7: u'Leblanc',
 8: u'Vladimir',
 9: u'FiddleSticks',
 10: u'Kayle',
 11: u'MasterYi',
 12: u'Alistar',
 13: u'Ryze',
 14: u'Sion',
 15: u'Sivir',
 16: u'Soraka',
 17: u'Teemo',
 18: u'Tristana',
 19: u'Warwick',
 20: u'Nunu',
 21: u'MissFortune',
 22: u'Ashe',
 23: u'Tryndamere',
 24: u'Jax',
 25: u'Morgana',
 26: u'Zilean',
 27: u'Singed',
 28: u'Evelynn',
 29: u'Twitch',
 30: u'Karthus',
 31: u'Chogath',
 32: u'Amumu',
 33: u'Rammus',
 34: u'Anivia',
 35: u'Shaco',
 36: u'DrMundo',
 37: u'Sona',
 38: u'Kassadin',
 39: u'Irelia',
 40: u'Janna',
 41: u'Gangplank',
 42: u'Corki',
 43: u'Karma',
 44: u'Taric',
 45: u'Veigar',
 48: u'Trundle',
 50: u'Swain',
 51: u'Caitlyn',
 53: u'Blitzcrank',
 54: u'Malphite',
 55: u'Katarina',
 56: u'Nocturne',
 57: u'Maokai',
 58: u'Renekton',
 59: u'JarvanIV',
 60: u'Elise',
 61: u'Orianna',
 62: 'Wukong',
 63: u'Brand',
 64: u'LeeSin',
 67: u'Vayne',
 68: u'R

In [7]:
# MATCH-v2.2
 
KEY = 'b00c992b-6fc1-4795-abcc-7db85c1b72fa'

REGION_ENDPOINT = "https://{0}.api.pvp.net/api/lol/{0}/"

import requests
def get_match(REGION, matchId, includeTimeline):
    """
    Retrieve match by match ID.
    """
    return requests.get(
        (REGION_ENDPOINT + "v2.2/match/{1}?"
         "api_key={2}&includeTimeline={3}").
        format(REGION, matchId, KEY, includeTimeline))

In [145]:
test_response = get_match('na', 2026610783, True).json()
#test_response = get_match('na', 2027658798, True)
print test_response
#print test_response.status_code
#print test_response.json()
#print test_response['queueType']

#def isClassicMatch(match):
#    j = match.json()
#    if j['mapId'] == 11 and j['matchMode'] == "CLASSIC" and j['matchType'] == 'MATCHED_GAME':
#        return True
#    else:
#        return False

#print isClassicMatch(test_response)

{u'queueType': u'RANKED_TEAM_5x5', u'matchVersion': u'5.23.0.239', u'platformId': u'NA1', u'season': u'PRESEASON2016', u'region': u'NA', u'matchId': 2026610783, u'mapId': 11, u'matchCreation': 1449099999881, u'teams': [{u'firstDragon': False, u'bans': [{u'pickTurn': 1, u'championId': 203}, {u'pickTurn': 3, u'championId': 41}, {u'pickTurn': 5, u'championId': 420}], u'firstInhibitor': False, u'baronKills': 0, u'firstRiftHerald': False, u'winner': True, u'firstBaron': False, u'riftHeraldKills': 1, u'firstBlood': True, u'teamId': 100, u'firstTower': False, u'vilemawKills': 0, u'inhibitorKills': 0, u'towerKills': 6, u'dominionVictoryScore': 0, u'dragonKills': 1}, {u'firstDragon': True, u'bans': [{u'pickTurn': 2, u'championId': 117}, {u'pickTurn': 4, u'championId': 16}, {u'pickTurn': 6, u'championId': 223}], u'firstInhibitor': False, u'baronKills': 0, u'firstRiftHerald': True, u'winner': False, u'firstBaron': False, u'riftHeraldKills': 1, u'firstBlood': False, u'teamId': 200, u'firstTower': 

In [117]:
valid_rank_games = ['RANKED_SOLO_5x5',  # Ranked Solo 5v5 games
                    'RANKED_PREMADE_3x3',  # Ranked Premade 3v3 games
                    'RANKED_PREMADE_5x5',  # Ranked Premade 5v5 games
                    'RANKED_TEAM_3x3',  # Ranked Team 3v3 games
                    'RANKED_TEAM_5x5']  # Ranked Team 5v5 games



['RANKED_SOLO_5x5', 'RANKED_PREMADE_3x3', 'RANKED_PREMADE_5x5', 'RANKED_TEAM_3x3', 'RANKED_TEAM_5x5']


In [151]:
1+1

2

In [ ]:
#test_response = get_match(region, 1516266210, True).json()
#test_response = get_match('na', 1516266210, True)
#print type(test_response)
#print test_response.status_code
#print test_response.json()

import json
import time

# Constants
STARTING_MATCH_ID = 2026690021
N_MATCHES = 5
REGION = 'na'
STATUS_OK = 200
STATUS_RATE_LIMIT_EXCEEDED = 429
valid_rank_games = ['RANKED_SOLO_5x5',  # Ranked Solo 5v5 games
                    'RANKED_PREMADE_3x3',  # Ranked Premade 3v3 games
                    'RANKED_PREMADE_5x5',  # Ranked Premade 5v5 games
                    'RANKED_TEAM_3x3',  # Ranked Team 3v3 games
                    'RANKED_TEAM_5x5']  # Ranked Team 5v5 games

# Functions that check status of the data pulls
def isValid(match):
    if match.status_code != STATUS_OK:
        return False
    return True

def isClassicMatch(match):
    j = match.json()
    if j['mapId'] == 11 and j['matchMode'] == "CLASSIC" and j['matchType'] == 'MATCHED_GAME':
        return True
    else:
        return False

def isRankMatch(match):
    j = match.json()
    if j['queueType'] in valid_rank_games:
        return True
    else:
        return False
                    
def rateLimitExceeded(match):
    if int(match.status_code) == STATUS_RATE_LIMIT_EXCEEDED:
        return True
    return False

counter = 0
i = 0

for i in range(10000):
    #print 'index is %d' % i
    m = get_match(REGION, i + STARTING_MATCH_ID, True)
    if not rateLimitExceeded(m):
        if isValid(m) and isClassicMatch(m) and isRankMatch(m):
            f = f = open("dump/" + str(i + STARTING_MATCH_ID)+".json", 'w+')
            #f = open("test_dump/"+str(i + STARTING_MATCH_ID)+".json", 'w+')
            f.write(m.text.encode('utf-8'))
            counter += 1
            print 'counter is %d' % counter
            f.close()
            i += 1
            #print 'index is %d' % i
    else:
        print("sleeping")
        #print 'index is %d' % i
        time.sleep(10)
        i += 1

sleeping
counter is 1
sleeping
counter is 2
counter is 3
counter is 4
sleeping
counter is 5
counter is 6
counter is 7
sleeping
sleeping
sleeping

In [ ]:
# 1916266995
# 1816266295 <- this game took place on May 7, 2015 BUT THIS IS ARAM
# 1516266297 <- game from Aug 31, 2015

# Constants
STARTING_MATCH_ID = 1516266210
N_MATCHES = 1
REGION = "na"
STATUS_OK = 200
STATUS_RATE_LIMIT_EXCEEDED = 429

# Functions that check status of the data pulls
def isValid(match):
    if match.status_code != STATUS_OK:
        return False
    return True

def isClassicMatch(match):
    j = match.json()
    return j['matchMode'] == "CLASSIC"

def rateLimitExceeded(match):
    if int(match.status_code) == STATUS_RATE_LIMIT_EXCEEDED:
        return True
    return False

#test_response = get_match(region, 1516266210, True).json()
test_response = get_match(REGION, 1516266210, True)
print type(test_response)
print test_response.status_code

counter = 0
i = 0
while counter < N_MATCHES:
    try: # get_match might have a connection time-out
        m = get_match(REGION, i + STARTING_MATCH_ID, True)
        if not rateLimitExceeded(m):
            if isValid(m) and isClassicMatch(m):
                try:
                    f = open("dump/"+str(i + STARTING_MATCH_ID)+".json", 'w+')
                    f.write(m.text)
                    counter += 1
                    print(counter)
                    f.close()
                except:
                    pass
            i += 1
            print i
            print rateLimitExceeded(m)
        else:
            print("sleeping")
            print i
            time.sleep(1)
    except:
        pass


<class 'requests.models.Response'>
200
1
False
2
False
3
False
4
False
5
False
6
False
7
False
8
False
9
False
sleeping
9
sleeping
9
sleeping
9
sleeping
9
sleeping
9
sleeping
9
sleeping
9
sleeping
9
sleeping
9
sleeping
9
sleeping
9
sleeping
9
sleeping
9
sleeping
9
sleeping
9
sleeping
9
10
False
11
False
12
False
13
False
14
False
15
False
16
False
17
False
18
False
19
False
sleeping
19
sleeping
19
sleeping
19
sleeping
19
sleeping
19
sleeping
19
sleeping
19
sleeping
19
sleeping
19
sleeping
19
sleeping
19
sleeping
19
sleeping
19
sleeping
19
sleeping
19
sleeping
19
20
False
21
False
22
False
23
False
24
False
25
False
26
False
27
False
28
False
29
False
sleeping
29
sleeping
29
sleeping
29
sleeping
29
sleeping
29
sleeping
29
sleeping
29
30
False
sleeping
30
sleeping
30
sleeping
30
sleeping
30
sleeping
30
sleeping
30
sleeping
30
31
False
32
False
33
False
34
False
35
False
36
False
37
False
38
False
39
False
40
False
sleeping
40
sleeping
40
sleeping
40
sleeping
40
sleeping
40
sleeping
40
sl

In [33]:
print "dump/"+str(i + STARTING_MATCH_ID)+".json"

dump/1516266216.json


In [ ]:
# practicing dumping json files

import json
fd = open("test_dump.json","w")
json.dump(test_response, fd)
fd.close()

In [4]:

# our region of interest is North America
region = 'na'

# functions to convert date to epoch time
#https://na.api.pvp.net/api/lol/na/v2.2/matchlist/by-summoner/28941311?championIds=267&rankedQueues=RANKED_SOLO_5x5&seasons=SEASON2015&beginTime=1438448045&endTime=1446400445&beginIndex=1&endIndex=5&api_key=b00c992b-6fc1-4795-abcc-7db85c1b72fa


#import rawpi
import json
import time


# Constants
STARTING_MATCH_ID = 1516166994
N_MATCHES = 10
REGION = "na"
STATUS_OK = 200
STATUS_RATE_LIMIT_EXCEEDED = 429


def isValid(match):
    if match.status_code != STATUS_OK:
        return False
    return True

def isClassicMatch(match):
    j = match.json()
    return j['matchMode'] == "CLASSIC"

def rateLimitExceeded(match):
    if int(match.status_code) == STATUS_RATE_LIMIT_EXCEEDED:
        return True
    return False


counter = 0
i = 0
while counter < N_MATCHES:
    try: # get_match might have a connection time-out
        m = get_match(REGION, i + STARTING_MATCH_ID, True)
        if not rateLimitExceeded(m):
            if isValid(m) and isClassicMatch(m):
                try:
                    f = open("dump/" + str(i + STARTING_MATCH_ID)+".json", 'w+')
                    f.write(m.text)
                    counter += 1
                    print(counter)
                    f.close()
                except:
                    pass
            i += 1
        else:
            print("sleeping")
            time.sleep(10)
    except:
        pass

KeyboardInterrupt: 